In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

load data

In [174]:
merged_df = pd.read_csv('data/main/data_set.csv')

In [ ]:
list(merged_df.columns)

rename for ols formular

In [175]:
merged_df = merged_df.rename(columns={'debt/equity_ratio_2019' : 'debtxtoxequity_ratio_2019', 
                                        'debt/asset_ratio_2019' : 'debtxtoxassest_ratio_2019',
                                        'debt/equity_ratio_2020' : 'debtxtoxequity_ratio_2020', 
                                        'debt/asset_ratio_2020' : 'debtxtoxassest_ratio_2020',
                                        'debt/equity_ratio_2021' : 'debtxtoxequity_ratio_2021', 
                                        'debt/asset_ratio_2021' : 'debtxtoxassest_ratio_2021',
                                        })

In [176]:
ratios = ['cash_ratio','quick_ratio','current_ratio','debtxtoxequity_ratio','equity_ratio','debtxtoxassest_ratio']
types = ['aid_grant','aid_loan']
results_dict = {}

In [385]:
#loop over year
for year in [2020,2021]:

    for id, type in enumerate(types, start=1):
        type_inverse_year = types[id-2] + '_' + str(year)
        type_year = type + '_' + str(year)
        data_unconfounded = merged_df[merged_df[type_inverse_year].isnull()]

        #loop over ratio
        if year == 2021: # Remove companies that received other aid due to possible confounding
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_grant'+'_'+ str(year-1))].isnull()] ###
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_loan'+'_'+ str(year-1))].isnull()] ###
        
        results = {}
        

        for ratio in ratios:
            ratio_year_prior = ratio + '_' + str(year-1)
            ratio_year = ratio + '_' + str(year)

            data_dd = data_unconfounded[[ratio_year_prior , ratio_year, type_year]]
            data_dd['treatment_group'] = np.where(data_dd[type_year].isna() , 0, 1)
            data_dd = data_dd.drop(type_year, axis=1)
            data_dd.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_dd = data_dd.dropna()

            # data before the treatment
            df_before = data_dd[[ratio_year_prior, 'treatment_group']]
            df_before['t'] = 0
            df_before.columns = [ratio, 'treatment_group', 'time']

            # data after the treatment
            df_after = data_dd[[ratio_year, 'treatment_group']]
            df_after['t'] = 1
            df_after.columns = [ratio, 'treatment_group', 'time']

            # data for regression
            df_reg = pd.concat([df_before, df_after])

            # create the interaction 
            df_reg['interaction'] = df_reg.treatment_group * df_reg.time

            # drop extrem outliers
            df_reg = df_reg[df_reg[ratio] > 0 ]
            df_reg = df_reg.drop(df_reg[df_reg[ratio] > df_reg[ratio].quantile(.95)].index)
            

            from statsmodels.formula.api import ols

            formular = ratio + ' ~ treatment_group + time + interaction' 
            ols = ols(formular, data=df_reg).fit()

            cov = ols.params['interaction'].round(4)
            pv =  ols.pvalues['interaction'].round(3)
            
            
            if pv >= 0.1:
                pv = ("%.3f" % pv)           
                pv = "\space\space\space\space("+str(pv)+")"
                #cov = str(cov)+'***'
            elif (pv < 0.1) and ((pv > 0.05)):
                pv = ("%.3f" % pv)           
                pv = "*\space\space\space("+str(pv)+")"
                #cov = str(cov)+'** '  
            elif (pv < 0.05) and ((pv > 0.01)):
                pv = ("%.3f" % pv)           
                pv = "**\space\space("+str(pv)+")"
                # cov = str(cov)+'*  '
            elif pv < 0.01:
                pv = ("%.3f" % pv)           
                pv = "***\space("+str(pv)+")"
                #cov = str(cov)+'  '
            
            #pv = ("%.3f" % pv)           
            #pv = "("+str(pv)+")"
            #print(' '.join((str(cov),pv)))

            ratio = ratio.replace("_", " ").replace("x", " ")
            results[ratio] = ''.join((str(cov),pv))

        results_dict[type_year] = results

format output

In [382]:
df_results = pd.DataFrame.from_dict(results_dict, orient='index')
df_results.index.name='aid_instrument_year'
df_results.reset_index(inplace=True)
new_col = df_results['aid_instrument_year'].str.split('_',expand=True)
df_results['instrument'] = new_col[1]
df_results['year'] = new_col[2]
df_results.drop(columns =["aid_instrument_year"], inplace = True)
df_results.set_index(['year', 'instrument'],inplace=True)
df_results

cash ratio  \
year instrument                                          
2020 grant       -0.006\space\space\space\space(0.521)   
     loan                        0.065***\space(0.000)   
2021 grant                       0.093***\space(0.000)   
     loan                       0.0388***\space(0.000)   

                                            quick ratio  \
year instrument                                           
2020 grant       -0.0871\space\space\space\space(0.457)   
     loan                   0.0905**\space\space(0.029)   
2021 grant        0.0357\space\space\space\space(0.853)   
     loan              0.2039*\space\space\space(0.094)   

                                          current ratio  \
year instrument                                           
2020 grant       -0.1636\space\space\space\space(0.204)   
     loan                        0.1518***\space(0.001)   
2021 grant       -0.0279\space\space\space\space(0.895)   
     loan              0.2657*\space\space\space(0.072)   

                                   debt to equity ratio  \
year instrument                                           
2020 grant                  0.5661**\space\space(0.041)   
     loan                        1.0924***\space(0.000)   
2021 grant       -0.3862\space\space\space\space(0.126)   
     loan              0.7023*\space\space\space(0.070)   

                                           equity ratio  \
year instrument                                           
2020 grant       -0.0095\space\space\space\space(0.497)   
     loan                       -0.0575***\space(0.000)   
2021 grant        0.0136\space\space\space\space(0.405)   
     loan                       -0.0453***\space(0.002)   

                              debt to assest ratio  
year instrument                                     
2020 grant             0.0366**\space\space(0.024)  
     loan                   0.0764***\space(0.000)  
2021 grant       -0.0319*\space\space\space(0.083)  
     loan                   0.0558***\space(0.002)

save data frame to latex

In [378]:
with open('/Users/marco/Library/CloudStorage/OneDrive-HertieSchool/Thesis/GitHub/thesis/paper/Tables/table_did.tex', 'w') as f:
     f.write(df_results.transpose().stack().to_latex(index=True, bold_rows=True, column_format='llrr',multirow=True ,formatters={'white-space': 'pre'}, escape=False))

/var/folders/nr/94zdnpbn2xj3pwc_d9lfk0cw0000gn/T/ipykernel_48019/2348292745.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_results.transpose().stack().to_latex(index=True, bold_rows=True, column_format='llrr',multirow=True ,formatters={'white-space': 'pre'}, escape=False))


In [525]:
for year in [2020,2021]:

    for id, type in enumerate(types, start=1):
        type_inverse_year = types[id-2] + '_' + str(year)
        type_year = type + '_' + str(year)
        data_unconfounded = merged_df[merged_df[type_inverse_year].isnull()]

        #loop over ratio
        if year == 2021: # Remove companies that received other aid due to possible confounding
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_grant'+'_'+ str(year-1))].isnull()] ###
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_loan'+'_'+ str(year-1))].isnull()] ###
        
        results = {}
        results_sub = {}

        for ratio in ratios:
            ratio_year_prior = ratio + '_' + str(year-1)
            ratio_year = ratio + '_' + str(year)

            data_dd = data_unconfounded[[ratio_year_prior , ratio_year, type_year]]
            data_dd['treatment_group'] = np.where(data_dd[type_year].isna() , 0, 1)
            data_dd = data_dd.drop(type_year, axis=1)
            data_dd.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_dd = data_dd.dropna()

            # data before the treatment
            df_before = data_dd[[ratio_year_prior, 'treatment_group']]
            df_before['t'] = 0
            df_before.columns = [ratio, 'treatment_group', 'time']

            # data after the treatment
            df_after = data_dd[[ratio_year, 'treatment_group']]
            df_after['t'] = 1
            df_after.columns = [ratio, 'treatment_group', 'time']

            # data for regression
            df_reg = pd.concat([df_before, df_after])

            # create the interaction 
            df_reg['interaction'] = df_reg.treatment_group * df_reg.time

            # drop extrem outliers
            df_reg = df_reg[df_reg[ratio] > 0 ]
            df_reg = df_reg.drop(df_reg[df_reg[ratio] > df_reg[ratio].quantile(.95)].index)
            

            from statsmodels.formula.api import ols

            formular = ratio + ' ~ treatment_group + time + interaction' 
            ols = ols(formular, data=df_reg).fit()

            cov = ols.params['interaction'].round(4)
            pv =  ols.pvalues['interaction'].round(3)

            cov_time = ols.params['time'].round(4)
            pv_time =  ols.pvalues['time'].round(3)

            cov_treat = ols.params['treatment_group'].round(4)
            pv_treat =  ols.pvalues['treatment_group'].round(3)
            

            if pv >= 0.1:
                pv = ("%.3f" % pv)           
                pv = " ("+str(pv)+")"
            elif (pv < 0.1) and (pv > 0.05):
                pv = ("%.3f" % pv)           
                pv = "* ("+str(pv)+")"
            elif (pv < 0.05) and (pv > 0.01):
                pv = ("%.3f" % pv)           
                pv = "** ("+str(pv)+")"
            elif pv < 0.01:
                pv = ("%.3f" % pv)           
                pv = "*** ("+str(pv)+")"

            if pv_time >= 0.1:
                pv_time = ("%.3f" % pv_time)           
                pv_time = " ("+str(pv_time)+")"
            elif (pv_time < 0.1) and (pv_time > 0.05):
                pv_time = ("%.3f" % pv_time)           
                pv_time = "* ("+str(pv_time)+")"
            elif (pv_time < 0.05) and (pv_time > 0.01):
                pv_time = ("%.3f" % pv_time)           
                pv_time = "** ("+str(pv_time)+")"
            elif pv_time < 0.01:
                pv_time = ("%.3f" % pv_time)           
                pv_time = "*** ("+str(pv_time)+")"

            if pv_treat >= 0.1:
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = " ("+str(pv_treat)+")"
            elif (pv_treat < 0.1) and (pv_treat > 0.05):
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = "* ("+str(pv_treat)+")"
            elif (pv_treat < 0.05) and (pv_treat > 0.01):
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = "** ("+str(pv_treat)+")"
            elif pv_treat < 0.01:
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = "*** ("+str(pv_treat)+")"

            ratio = ratio.replace("_", " ").replace("x", " ")
            results_sub['aid'] = ''.join((str(cov_treat),pv_treat))
            results_sub['post'] = ''.join((str(cov_time),pv_time))
            results_sub['aid*post'] =''.join((str(cov),pv))
            results[ratio] = results_sub

        results_dict[type_year] = results

In [526]:
reformed_dict = {}
for outerKey, innerDict in results_dict.items():
    for innerKey, values in innerDict.items():
        reformed_dict[(outerKey,
                       innerKey)] = values
df_results_all = pd.DataFrame(reformed_dict)
df_results_all
  

aid_grant_2020                                      \
                cash ratio       quick ratio     current ratio   
aid        -0.0069 (0.551)   -0.0069 (0.551)   -0.0069 (0.551)   
post        -0.005 (0.300)    -0.005 (0.300)    -0.005 (0.300)   
aid*post  0.0366** (0.024)  0.0366** (0.024)  0.0366** (0.024)   

                                                                      \
         debt to equity ratio      equity ratio debt to assest ratio   
aid           -0.0069 (0.551)   -0.0069 (0.551)      -0.0069 (0.551)   
post           -0.005 (0.300)    -0.005 (0.300)       -0.005 (0.300)   
aid*post     0.0366** (0.024)  0.0366** (0.024)     0.0366** (0.024)   

              aid_loan_2020                                        \
                 cash ratio        quick ratio      current ratio   
aid       0.0831*** (0.000)  0.0831*** (0.000)  0.0831*** (0.000)   
post         -0.005 (0.251)     -0.005 (0.251)     -0.005 (0.251)   
aid*post  0.0764*** (0.000)  0.0764*** (0.000)  0.0764*** (0.000)   

                               ...    aid_grant_2021                       \
         debt to equity ratio  ...     current ratio debt to equity ratio   
aid         0.0831*** (0.000)  ...  -0.029** (0.026)     -0.029** (0.026)   
post           -0.005 (0.251)  ...    0.0164 (0.323)       0.0164 (0.323)   
aid*post    0.0764*** (0.000)  ...  -0.0319* (0.083)     -0.0319* (0.083)   

                                                     aid_loan_2021  \
              equity ratio debt to assest ratio         cash ratio   
aid       -0.029** (0.026)     -0.029** (0.026)  0.0962*** (0.000)   
post        0.0164 (0.323)       0.0164 (0.323)     0.0151 (0.288)   
aid*post  -0.0319* (0.083)     -0.0319* (0.083)  0.0558*** (0.002)   

                                                                     \
                quick ratio      current ratio debt to equity ratio   
aid       0.0962*** (0.000)  0.0962*** (0.000)    0.0962*** (0.000)   
post         0.0151 (0.288)     0.0151 (0.288)       0.0151 (0.288)   
aid*post  0.0558*** (0.002)  0.0558*** (0.002)    0.0558*** (0.002)   

                                                  
               equity ratio debt to assest ratio  
aid       0.0962*** (0.000)    0.0962*** (0.000)  
post         0.0151 (0.288)       0.0151 (0.288)  
aid*post  0.0558*** (0.002)    0.0558*** (0.002)  

[3 rows x 24 columns]

In [527]:
df_results_all = pd.DataFrame.from_dict(reformed_dict, orient='index')

df_results_all.reset_index(inplace=True)
df_results_all = df_results_all.rename(columns={'level_0' : 'aid_type_year', 
                                        'level_1' : 'ratio'})

#df_results_all.index.name='aid_instrument_year'

new_col = df_results_all['aid_type_year'].str.split('_',expand=True)
df_results_all['type'] = new_col[1]
df_results_all['year'] = new_col[2]
df_results_all.drop(columns =["aid_type_year"], inplace = True)
df_results_all.set_index(['type','year', 'ratio'],inplace=True)
#df_results_all = df_results_all.transpose().stack()
#df_results_all.index = df_results_all.index.set_names(['instrument', 'year'])
df_results_all

aid            post  \
type  year ratio                                                     
grant 2020 cash ratio              -0.0069 (0.551)  -0.005 (0.300)   
           quick ratio             -0.0069 (0.551)  -0.005 (0.300)   
           current ratio           -0.0069 (0.551)  -0.005 (0.300)   
           debt to equity ratio    -0.0069 (0.551)  -0.005 (0.300)   
           equity ratio            -0.0069 (0.551)  -0.005 (0.300)   
           debt to assest ratio    -0.0069 (0.551)  -0.005 (0.300)   
loan  2020 cash ratio            0.0831*** (0.000)  -0.005 (0.251)   
           quick ratio           0.0831*** (0.000)  -0.005 (0.251)   
           current ratio         0.0831*** (0.000)  -0.005 (0.251)   
           debt to equity ratio  0.0831*** (0.000)  -0.005 (0.251)   
           equity ratio          0.0831*** (0.000)  -0.005 (0.251)   
           debt to assest ratio  0.0831*** (0.000)  -0.005 (0.251)   
grant 2021 cash ratio             -0.029** (0.026)  0.0164 (0.323)   
           quick ratio            -0.029** (0.026)  0.0164 (0.323)   
           current ratio          -0.029** (0.026)  0.0164 (0.323)   
           debt to equity ratio   -0.029** (0.026)  0.0164 (0.323)   
           equity ratio           -0.029** (0.026)  0.0164 (0.323)   
           debt to assest ratio   -0.029** (0.026)  0.0164 (0.323)   
loan  2021 cash ratio            0.0962*** (0.000)  0.0151 (0.288)   
           quick ratio           0.0962*** (0.000)  0.0151 (0.288)   
           current ratio         0.0962*** (0.000)  0.0151 (0.288)   
           debt to equity ratio  0.0962*** (0.000)  0.0151 (0.288)   
           equity ratio          0.0962*** (0.000)  0.0151 (0.288)   
           debt to assest ratio  0.0962*** (0.000)  0.0151 (0.288)   

                                          aid*post  
type  year ratio                                    
grant 2020 cash ratio             0.0366** (0.024)  
           quick ratio            0.0366** (0.024)  
           current ratio          0.0366** (0.024)  
           debt to equity ratio   0.0366** (0.024)  
           equity ratio           0.0366** (0.024)  
           debt to assest ratio   0.0366** (0.024)  
loan  2020 cash ratio            0.0764*** (0.000)  
           quick ratio           0.0764*** (0.000)  
           current ratio         0.0764*** (0.000)  
           debt to equity ratio  0.0764*** (0.000)  
           equity ratio          0.0764*** (0.000)  
           debt to assest ratio  0.0764*** (0.000)  
grant 2021 cash ratio             -0.0319* (0.083)  
           quick ratio            -0.0319* (0.083)  
           current ratio          -0.0319* (0.083)  
           debt to equity ratio   -0.0319* (0.083)  
           equity ratio           -0.0319* (0.083)  
           debt to assest ratio   -0.0319* (0.083)  
loan  2021 cash ratio            0.0558*** (0.002)  
           quick ratio           0.0558*** (0.002)  
           current ratio         0.0558*** (0.002)  
           debt to equity ratio  0.0558*** (0.002)  
           equity ratio          0.0558*** (0.002)  
           debt to assest ratio  0.0558*** (0.002)

In [531]:
with open('/Users/marco/Library/CloudStorage/OneDrive-HertieSchool/Thesis/GitHub/thesis/paper/Tables/table_did_all.tex', 'w') as f:
     f.write(df_results_all.to_latex())

/var/folders/nr/94zdnpbn2xj3pwc_d9lfk0cw0000gn/T/ipykernel_48019/3209959191.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_results_all.to_latex())
